## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
# python-dotenv 패키지: 환경 변수를 .env 파일에서 로드하는 라이브러리
# - 보안이 필요한 API 키 등을 관리하는데 사용
# - pip install python-dotenv로 설치 가능
from dotenv import load_dotenv

# load_dotenv(): .env 파일의 환경 변수를 현재 실행 환경에 로드하는 함수
# - 반환값: 성공 시 True, 실패 시 False
load_dotenv()


True

`(2) 기본 라이브러리`

In [3]:
# os 패키지: 운영체제와 상호작용하는 기능을 제공하는 파이썬 표준 라이브러리
# - 파일/디렉토리 경로 조작, 환경변수 접근 등에 사용
# - 파이썬 기본 내장 라이브러리로 별도 설치 불필요
import os

# glob 패키지: 파일 경로 패턴 매칭을 지원하는 파이썬 표준 라이브러리 
# - 와일드카드(*,?)를 사용한 파일 검색에 활용
# - 파이썬 기본 내장 라이브러리로 별도 설치 불필요
from glob import glob

## 2. LangChain RAG 구현

### 2.1 문서 로더 (Document Loaders)

In [4]:
# 텍스트 파일 목록 가져오기
# glob() 함수를 사용하여 'data' 디렉토리에서 '_KR.txt'로 끝나는 모든 텍스트 파일을 검색
# os.path.join()을 사용하여 운영체제에 맞는 경로 구분자로 경로를 생성
# - 'data': 검색할 디렉토리 경로
# - '*_KR.txt': 파일명 패턴 (_KR.txt로 끝나는 모든 파일)

# 두 코드는 동일한 결과를 반환합니다.(glob('data/*_KR.txt') 와 동일)
# 1. os.path.join('data', '*_KR.txt')는 운영체제에 맞는 경로 구분자를 사용
# - Windows: 'data\*_KR.txt' 
# - Unix/Linux/Mac: 'data/*_KR.txt'

# 2. 'data/*_KR.txt'는 직접 경로 구분자(/)를 사용
# - 대부분의 운영체제에서 호환되지만 Windows에서 문제가 될 수 있음

# 더 안전한 방식인 os.path.join() 사용을 권장
txt_files = glob(os.path.join('data', '*_KR.txt'))


print(txt_files)

['data/리비안_KR.txt', 'data/테슬라_KR.txt']


In [5]:
# 첫 번째 텍스트 파일 내용 확인
# langchain_community.document_loaders 패키지: LangChain에서 제공하는 다양한 문서 로더 모듈
# - TextLoader: 텍스트 파일(.txt)을 로드하는 기본 로더 클래스
# - 파일을 읽어서 LangChain Document 객체로 변환
# - pip install langchain-community로 설치 필요
from langchain_community.document_loaders import TextLoader

# Document 객체는 LangChain의 핵심 데이터 구조입니다.
# - page_content: 문서의 실제 텍스트 내용을 저장
# - metadata: 파일 경로, 생성 날짜 등 문서의 메타데이터를 딕셔너리 형태로 저장
# TextLoader는 텍스트 파일을 읽어서 Document 객체로 변환합니다.
loader = TextLoader(txt_files[0], encoding='utf-8')
data = loader.load()
len(data)

1

In [8]:
# data type 확인
type(data)

list

In [9]:
# page_content를 출력
print(data[0].page_content)

리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다.

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.



In [10]:
# metadata를 출력
print(data[0].metadata)

{'source': 'data/리비안_KR.txt'}


In [6]:
# Load all files from a directory
# tqdm: 진행 상태 표시줄을 제공하는 라이브러리
# - 반복문의 진행 상황을 시각적으로 보여줌
# - 남은 시간, 처리 속도 등의 정보를 제공
# - pip install tqdm으로 설치 필요
from tqdm import tqdm

data = []

for text_file in tqdm(txt_files):
    
    # encoding 파라미터의 기본값은 'utf-8'입니다
    loader = TextLoader(text_file, encoding='utf-8')
    data += loader.load()

    from time import sleep
    sleep(0.5)

len(data)

100%|██████████| 2/2 [00:01<00:00,  1.98it/s]


2

In [15]:
print(data[0].page_content)

리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다.

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.



In [18]:
print(data[1].page_content)

테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.

2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 세계 전기차 시장의 약 12.9%를 차지했습니다.



In [17]:
type(data[0])

langchain_core.documents.base.Document

### 2.2 텍스트 분할기 (Text Splitters)

In [19]:
# 각 문서의 글자수를 계산
char_count = [len(doc.page_content) for doc in data]

print(char_count)

[514, 451]


In [7]:
# langchain_text_splitters: LangChain의 텍스트 분할 기능을 제공하는 패키지
# CharacterTextSplitter: 문자 단위로 텍스트를 분할하는 클래스
# - chunk_size로 지정된 크기만큼 텍스트를 분할
# - chunk_overlap으로 청크 간 중복 영역 설정 가능
# - separator로 분할 기준이 되는 구분자 지정 가능
# - pip install langchain-text-splitters로 설치 필요
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=250,        # 청크 크기  
    chunk_overlap=50,      # 청크 중 중복되는 부분 크기
    separator='\n\n',      # 청크를 나눌 때 사용할 구분자
)

# text_splitter.split_documents() 함수는 Document 객체 리스트를 입력받아 더 작은 청크로 분할합니다.
# - data: Document 객체 리스트 (입력)
# - 반환값: 분할된 Document 객체 리스트 
# - chunk_size, chunk_overlap, separator 설정에 따라 텍스트를 분할
texts = text_splitter.split_documents(data)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

Created a chunk of size 282, which is longer than the specified 250
Created a chunk of size 268, which is longer than the specified 250


생성된 텍스트 청크 수: 5
각 청크의 길이: [175, 282, 52, 268, 180]


- CharacterTextSplitter는 단순히 문자 수로만 텍스트를 자르지 않습니다. 
- 기본적으로 문장이나 단락의 경계를 존중하려고 합니다. 
- 따라서 문장이나 단락의 끝부분에서 지정된 chunk_size를 약간 초과할 수 있습니다.

In [21]:
print("첫 번째 청크의 내용:", texts[0].page_content)

첫 번째 청크의 내용: 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.


In [23]:
print("두 번째 청크의 내용:", texts[1].page_content)

두 번째 청크의 내용: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다.


In [24]:
print("세 번째 청크의 내용:", texts[2].page_content)

세 번째 청크의 내용: 리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.


In [29]:
print("네 번째 청크의 내용:", texts[3].page_content)
print()
print("다섯 번째 청크의 내용:", texts[4].page_content)

네 번째 청크의 내용: 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.

다섯 번째 청크의 내용: 2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 세계 전기차 시장의 약 12.9%를 차지했습니다.


In [30]:
# 겹치는 부분을 출력
print("첫 번째 청크의 최종 50글자:", texts[0].page_content[-50:])
print("두 번째 청크의 처음 50글자:", texts[1].page_content[:50])

첫 번째 청크의 최종 50글자: 했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.
두 번째 청크의 처음 50글자: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 


In [8]:
text_splitter = CharacterTextSplitter(
    chunk_size=250,        # 청크 크기 (separator=''인 경우에도 문장/단락 경계 존중)
    chunk_overlap=50,      # 청크 중 중복되는 부분 크기  
    separator='',          # 빈 문자열로 설정해도 문장/단락 경계를 기준으로 분할
)

texts = text_splitter.split_documents(data)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

생성된 텍스트 청크 수: 6
각 청크의 길이: [250, 250, 112, 250, 250, 50]


In [33]:
# 겹치는 부분을 출력
print("첫 번째 청크의 최종 50글자:", texts[0].page_content[-50:])
print("두 번째 청크의 처음 50글자:", texts[1].page_content[:50])

첫 번째 청크의 최종 50글자: 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했
두 번째 청크의 처음 50글자: 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했


### 2.3 임베딩 모델 (Embeddings)

In [9]:
# langchain_openai 패키지에서 OpenAIEmbeddings 클래스를 가져옵니다.
# OpenAIEmbeddings는 OpenAI의 임베딩 모델을 사용하여 텍스트를 벡터로 변환하는 기능을 제공합니다.
# 이를 통해 텍스트의 의미를 수치화된 벡터 형태로 표현할 수 있습니다.
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름을 지정 가능 
)
sample_text = "테슬라 창업자는 누구인가요?"
vector = embeddings.embed_query(sample_text)
print(f"임베딩 벡터의 차원: {len(vector)}")

임베딩 벡터의 차원: 1536


### 2.4 벡터 저장소 (Vector Stores)

In [35]:
texts[0]

Document(metadata={'source': 'data/리비안_KR.txt'}, page_content='리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했')

In [10]:
from langchain_chroma import Chroma


vectorstore = Chroma.from_documents(
    documents=texts, 
    embedding=embeddings,
    collection_name="chroma_test", 
    persist_directory="./chroma_db",
    )
print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 30


In [37]:
vectorstore.get()

{'ids': ['88cbedcb-dad9-4c37-a9c4-5e87a596a13f',
  '905cd047-c314-4e96-b0ce-254eb6436bdb',
  'd75ed655-96ce-4543-ab42-d71f0304e62b',
  'a0eb95ca-4709-4095-9976-df24a51d6d91',
  '27a4376f-30f6-4054-a61a-466f5d422d40',
  '7b0e8506-4d36-410f-a4d7-14e3ee369d47',
  '474cd70a-5d57-425b-a77c-890a2121ceba',
  '35753b3c-3443-4838-b773-c0041faa2561',
  'd3e8c3a3-dcba-4f7f-8a6d-d4e23887cf39',
  '48a35757-e52b-4fbd-8361-12610b78ac9f',
  '45a0101a-11a8-45e5-bef2-70a88a4b1449',
  '4d1814e8-a970-4272-bb9e-e6edffb17989'],
 'embeddings': None,
 'documents': ['리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했',
  '이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 

In [11]:
# 문서 검색
query = "테슬라 창업자는 누구인가요?"

# vectorstore.similarity_search() 메서드는 입력된 쿼리와 가장 유사한 문서들을 검색합니다.
# 이 메서드는 쿼리 텍스트를 임베딩 벡터로 변환한 후,
# 벡터 저장소에 있는 모든 문서들의 임베딩 벡터와 코사인 유사도를 계산하여
# 가장 유사한 문서들을 반환합니다.
result = vectorstore.similarity_search(query, k=2)

print(f"검색 결과의 길이: {len(result)}")
print(f"검색 결과의 첫 번째 문서: {result[0].page_content[:100]}...")

검색 결과의 길이: 2
검색 결과의 첫 번째 문서: 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는...


### 2.5 검색기 (Retrievers)
   - 질의에 관련된 문서를 검색하는 컴포넌트

In [12]:
# vectorstore를 retriever로 변환합니다
# as_retriever() 메서드를 사용하여 벡터 저장소를 검색기로 변환합니다
# search_kwargs={"k": 2}는 검색 시 상위 2개의 가장 관련성 높은 문서를 반환하도록 설정합니다
# retriever는 벡터 저장소에서 관련 문서를 검색하는 인터페이스입니다
# as_retriever() 메서드로 vectorstore를 retriever로 변환합니다
# search_kwargs={"k": 2}는 검색 시 상위 2개의 가장 유사한 문서를 반환하도록 설정합니다
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# retriever는 runnable 객체이므로 invoke() 메서드를 사용할 수 있습니다
relevant_docs = retriever.invoke(query)
print(f"검색된 관련 문서 수: {len(relevant_docs)}")
print(f"첫 번째 관련 문서 내용 미리보기: {relevant_docs[0].page_content[:100]}...")

검색된 관련 문서 수: 2
첫 번째 관련 문서 내용 미리보기: 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는...


### 2.6 언어 모델 (LLMs)

In [13]:
# Context를 제공하지 않고 답변 생성 (Hallucination 발생 위험)

# langchain_openai 패키지에서 ChatOpenAI 클래스를 가져옵니다.
# ChatOpenAI는 OpenAI의 GPT 모델을 사용하여 대화형 AI를 구현하는 클래스입니다.
# 이를 통해 OpenAI의 API를 쉽게 사용할 수 있습니다.
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",  # 사용할 모델 이름을 지정 가능
    temperature=0,        # temperature는 0~1 사이의 값으로, 0에 가까울수록 일관된 답변을, 1에 가까울수록 다양하고 창의적인 답변을 생성합니다
    max_tokens=100,       # 생성할 최대 토큰 수
    )

# llm.invoke()는 ChatMessage 객체를 반환합니다
# ChatMessage는 AI의 응답을 구조화된 형식으로 담고 있는 객체입니다
# content: 실제 응답 텍스트
# type: 메시지 유형 (보통 'assistant')
# additional_kwargs: 추가 메타데이터
response = llm.invoke("테슬라 창업자는 누구인가요?")
print(f"LLM 응답: {response}")

LLM 응답: content='테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 성장에 큰 영향을 미쳤습니다. 이후 그는 테슬라의 가장 잘 알려진 얼굴이' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 17, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None} id='run-07a8064a-3de5-46b9-898b-583dfc170b11-0' usage_metadata={'input_tokens': 17, 'output_tokens': 100, 'total_tokens': 117}


In [45]:
# Context를 제공하고 답변 생성

query_with_context = f"""{relevant_docs[0].page_content}\n\n위 내용에 근거하여 다음 질문에 답변하세요. \n\n{query}"""
print(query_with_context)

테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010

위 내용에 근거하여 다음 질문에 답변하세요. 

테슬라 창업자는 누구인가요?


In [46]:
response = llm.invoke(query_with_context)
print(f"LLM 응답: {response.content}")

LLM 응답: 테슬라의 창업자는 마틴 에버하드와 마크 타페닝입니다.


### 2.7 전체 RAG 파이프라인 구성

In [47]:
# langchain hub - 프롬프트 템플릿, 체인 등 LangChain 컴포넌트를 공유하고 재사용할 수 있는 중앙 저장소
from langchain import hub

# ChatPromptTemplate - 챗봇 프롬프트를 쉽게 만들 수 있는 템플릿 클래스
from langchain_core.prompts import ChatPromptTemplate

# create_retrieval_chain - 검색과 응답 생성을 연결하는 체인을 만드는 함수
from langchain.chains import create_retrieval_chain

# create_stuff_documents_chain - 여러 문서를 하나의 컨텍스트로 결합하여 LLM에 전달하는 체인을 만드는 함수
from langchain.chains.combine_documents import create_stuff_documents_chain



# LLM 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 프롬프트 - 옵션 1 (허브 사용)
# https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
# retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

# 프롬프트 - 옵션 2 (직접 작성)
retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면, 
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")


# 체인 생성
# create_stuff_documents_chain 함수는 다음과 같은 역할을 합니다:
# - 여러 문서들을 하나의 컨텍스트로 결합(stuff)합니다
# - LLM과 프롬프트 템플릿을 받아서 문서 처리 체인을 생성합니다
# - 결합된 컨텍스트를 기반으로 질문에 답변하는 체인을 만듭니다
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

# create_retrieval_chain 함수는 다음과 같은 역할을 합니다:
# - retriever를 사용하여 관련 문서를 검색합니다
# - 검색된 문서를 combine_docs_chain에 전달하여 답변을 생성합니다
# - 검색과 답변 생성을 하나의 파이프라인으로 연결합니다
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke({"input": query})

In [48]:
# 결과 출력
print(response)

{'input': '테슬라 창업자는 누구인가요?', 'context': [Document(metadata={'source': 'data/테슬라_KR.txt'}, page_content='테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010'), Document(metadata={'source': 'data/테슬라_KR.txt'}, page_content='테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010')], 'answer': '테슬라의 창업자는 마틴 에버하드(CEO)와 마크 타페닝(CFO)입니다.'}


In [49]:
# 응답 객체의 키 확인
response.keys()

dict_keys(['input', 'context', 'answer'])

In [50]:
response["answer"]

'테슬라의 창업자는 마틴 에버하드(CEO)와 마크 타페닝(CFO)입니다.'

`[추가 설명] create_stuff_documents_chain`
- 검색된 문서들을 하나의 컨텍스트로 결합하고, 이를 바탕으로 질문에 답변하는 체인을 생성

In [51]:

# langchain_core.documents 모듈에서 Document 클래스를 가져옵니다.
# Document 클래스는 텍스트 콘텐츠와 메타데이터를 포함하는 문서를 표현하는 기본 클래스입니다.
# 주로 LLM 기반 애플리케이션에서 텍스트 데이터를 구조화하고 처리하는데 사용됩니다.
# langchain_core.documents 모듈의 Document 클래스는 다음과 같은 구조를 가집니다:
# - page_content: 문서의 실제 텍스트 내용을 저장하는 문자열 필드
# - metadata: 문서와 관련된 추가 정보(예: 출처, 작성일 등)를 저장하는 딕셔너리 필드
from langchain_core.documents import Document

stuff_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

documents = [
    Document(page_content="LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다."),
    Document(page_content="LangChain은 파이썬과 자바스크립트를 지원합니다.")
]
response = stuff_chain.invoke({
    "input": "LangChain은 어떤 언어를 지원하나요?",
    "context": documents
})
print(response)

LangChain은 파이썬과 자바스크립트를 지원합니다.


`[추가 설명] create_retrieval_chain`
- 질문에 관련된 문서를 검색하고, 검색된 문서를 바탕으로 답변을 생성하는 전체 RAG 파이프라인을 구축

In [54]:
# 검색 체인 생성
retrieval_chain = create_retrieval_chain(vectorstore.as_retriever(), stuff_chain)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = retrieval_chain.invoke({"input": query})

# 결과 출력
print(response)
print(response.keys())
print(response['answer'])

{'input': '테슬라 창업자는 누구인가요?', 'context': [Document(metadata={'source': 'data/테슬라_KR.txt'}, page_content='테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010'), Document(metadata={'source': 'data/테슬라_KR.txt'}, page_content='테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010'), Document(metadata={'source': 'data/테슬라_KR.txt'}, page_content='테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010

## 3. Gradio 챗봇

In [55]:
# Gradio는 머신러닝 모델을 위한 웹 인터페이스를 쉽게 만들 수 있게 해주는 파이썬 라이브러리입니다.
# 주요 특징:
# - 간단한 코드로 대화형 UI 생성 가능
# - 다양한 입출력 컴포넌트 제공 (텍스트, 이미지, 오디오 등)
# - 로컬 호스팅 및 Hugging Face Spaces 배포 지원
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke({"input": message})
    return response["answer"]


# Graiio 인터페이스 생성 
# Gradio의 ChatInterface 클래스를 사용하여 챗봇 인터페이스를 생성합니다
# - fn=answer_invoke: 사용자 입력을 처리할 콜백 함수를 지정합니다
# - title="QA Bot": 챗봇 UI의 제목을 설정합니다
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

/Users/gu.han/Documents/AI.WORK/RAG_Master/langchain_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/Users/gu.han/Documents/AI.WORK/RAG_Master/langchain_env/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [1]:
# Graiio 종료
demo.close()

NameError: name 'demo' is not defined